In [1]:
# !pip install webdriver_manager
# !apt-get update
# !apt install chromium-chromedriver --yes
# !pip install selenium
# !pip install ipywidgets

In [2]:
from tqdm import notebook
import time
import random
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [3]:
def chrome_setting():
  chrome_options = webdriver.ChromeOptions()
  chrome_options.add_argument('--headless')
  chrome_options.add_argument('--no-sandbox')
  chrome_options.add_argument('--disable-dev-shm-usage')
  driver = webdriver.Chrome('chromedriver', options=chrome_options)
  return driver
driver=chrome_setting()

In [4]:
# Crawling
def back_trans(text_data, lang):
    try:
        target_present = EC.presence_of_element_located((By.XPATH, '//*[@id="txtTarget"]'))
        driver.get('https://papago.naver.com/?sk=ko&tk='+lang+'&st='+text_data)
        time.sleep(random.uniform(0.2, 1))
        element=WebDriverWait(driver, 10).until(target_present)
        time.sleep(random.uniform(0.2, 1))
        backtrans = element.text
    except:
        print(text_data)
        return ""
    
    try:
        target_present = EC.presence_of_element_located((By.XPATH, '//*[@id="txtTarget"]'))
        driver.get('https://papago.naver.com/?sk='+lang+'&tk=ko&st='+backtrans)
        time.sleep(random.uniform(0.2, 1))
        element=WebDriverWait(driver, 10).until(target_present)
        time.sleep(random.uniform(0.2, 1))
        new_backtrans = element.text
        return new_backtrans
    except:
        print(text_data)
        return ""

In [5]:
# 테스트 영어/중국어/일본어 back-translation
print(back_trans("열린우리당을 흡수합당한 대통합민주신당과 민주당과의 합당때 합류하지 않은 친노직계 세력이 주축이 되어 2010년 1월 17일 창당한 정당이다.","en"))
print(back_trans("열린우리당을 흡수합당한 대통합민주신당과 민주당과의 합당때 합류하지 않은 친노직계 세력이 주축이 되어 2010년 1월 17일 창당한 정당이다.","zh-CN"))
print(back_trans("열린우리당을 흡수합당한 대통합민주신당과 민주당과의 합당때 합류하지 않은 친노직계 세력이 주축이 되어 2010년 1월 17일 창당한 정당이다.","ja"))

2010년 1월 17일 열린우리당을 흡수한 대통합민주신당과 민주당의 합당에 동참하지 않은 친노 세력이 주축이 되어 창당되었다.
열린우리당의 대통합민주신당과 민주당이 합당하면서 합류하지 않았던 친노 직계 세력을 주축으로 2010년 1월 17일 창당한 정당이다.
열린우리당을 흡수합병한 대통합민주신당과 민주당의 합당 때 합류하지 않았던 친노 직계세력이 주축이 돼 2010년 1월 17일 창당한 정당이다.


## 외국어인지 판별하는 함수

In [6]:
def find_foreign(start: int, end: int, sentence: str):
    count=0
    a = int(start, 16) 
    b = int(end, 16)
    return_sentence = ''
    for i, w in enumerate(sentence):
        if a <= ord(w) and ord(w) <= b:
            return True
    return False

## 품질 좋은 역번역 데이터 만들기
### 1. 중복 제거, 괄호 제거 전체 데이터를 가져온다
### 2. no_relation이 아니면서 + 외국어가 전혀 없으면서 + 각 엔티티가 문장에 한 번만 존재하는 경우에 역번역->엔티티가 유지될 경우에만 더해준다
### 3. 각 엔티티는 문장에서 한번만 나오므로 find로 index를 찾아 수정해준다.
### 4. (주) 특수문자 처리한다.
### 5. train/val split 하여 학습한다


## 특수문자도 바뀌는 경우가 있으니 이부분 더 고려해볼 것

In [7]:
import pandas as pd
import re
raw_df = pd.read_csv('/opt/ml/project2/level2_klue_nlp-level2-nlp-01/dataset/train/back_translation_input.csv')
raw_df
#/opt/ml/project2/level2_klue_nlp-level2-nlp-01/dataset/train/back_translation_input.csv

,id,sentence,subject_entity,object_entity,label,source
0,0,〈Something〉는 조지 해리슨이 쓰고 비틀즈가 1969년 앨범 《Abbey R...,"{'word': '비틀즈', 'start_idx': 24, 'end_idx': 26...","{'word': '조지 해리슨', 'start_idx': 13, 'end_idx':...",no_relation,wikipedia
1,1,호남이 기반인 바른미래당·대안신당·민주평화당이 우여곡절 끝에 합당해 민생당으로 재탄...,"{'word': '민주평화당', 'start_idx': 19, 'end_idx': ...","{'word': '대안신당', 'start_idx': 14, 'end_idx': 1...",no_relation,wikitree
2,2,K리그2에서 성적 1위를 달리고 있는 광주FC는 지난 26일 한국프로축구연맹으로부터...,"{'word': '광주FC', 'start_idx': 21, 'end_idx': 2...","{'word': '한국프로축구연맹', 'start_idx': 34, 'end_idx...",org:member_of,wikitree
3,3,균일가 생활용품점 아성다이소(대표 박정부)는 코로나19 바이러스로 어려움을 겪고 있...,"{'word': '아성다이소', 'start_idx': 10, 'end_idx': ...","{'word': '박정부', 'start_idx': 19, 'end_idx': 21...",org:top_members/employees,wikitree
4,4,1967년 프로 야구 드래프트 1순위로 요미우리 자이언츠에게 입단하면서 등번호는 8...,"{'word': '요미우리 자이언츠', 'start_idx': 22, 'end_id...","{'word': '1967', 'start_idx': 0, 'end_idx': 3,...",no_relation,wikipedia
...,...,...,...,...,...,...
32419,32466,"법포는 다시 최시형, 서병학, 손병희 직계인 북접과 다시 서장옥, 전봉준, 김개남을...","{'word': '최시형', 'start_idx': 7, 'end_idx': 9, ...","{'word': '손병희', 'start_idx': 17, 'end_idx': 19...",per:colleagues,wikipedia
32420,32467,완도군(군수 신우철)이 국토교통부에서 실시한 '2019 교통문화지수 실태조사'에서 ...,"{'word': '완도군', 'start_idx': 0, 'end_idx': 2, ...","{'word': '신우철', 'start_idx': 7, 'end_idx': 9, ...",org:top_members/employees,wikitree
32421,32468,"중앙일보, JTBC 회장을 지낸 이후 중앙홀딩스 회장, 재단법인 한반도평화만들기 이...","{'word': 'JTBC', 'start_idx': 6, 'end_idx': 9,...","{'word': '중앙홀딩스', 'start_idx': 21, 'end_idx': ...",no_relation,wikipedia
32422,32469,화순군(군수 구충곤)은 17일 동면의 이장 20여 명이 코로나 19 예방을 위해 버...,"{'word': '화순군', 'start_idx': 0, 'end_idx': 2, ...","{'word': '구충곤', 'start_idx': 7, 'end_idx': 9, ...",org:top_members/employees,wikitree


In [9]:
# df=raw_df.loc[0:6000] #혜빈
# df=raw_df.loc[6001:12000] #인진
# df=raw_df.loc[12001:18000] #용우 
# df=raw_df.loc[18001:24000] #준원
# df=raw_df.loc[24001:32423] #현정
df

,id,sentence,subject_entity,object_entity,label,source
24001,24024,또한 시리즈 구성은 《무한의 리바이어스》나 《허니와 클로버》 등으로 알려진 쿠로다 ...,"{'word': '코우가 윤', 'start_idx': 66, 'end_idx': ...","{'word': '만화가', 'start_idx': 62, 'end_idx': 64...",per:title,wikipedia
24002,24025,유진기업이 최첨단 기술이 적용된 스마트팩토리를 추진하는 배경은 유경선 유진그룹 회장...,"{'word': '유진그룹', 'start_idx': 39, 'end_idx': 4...","{'word': '유경선', 'start_idx': 35, 'end_idx': 37...",org:top_members/employees,wikitree
24003,24026,지난 19일 방송된 KBS '불후의 명곡2-전설을 노래하다' 배철수 편에서는 가수 ...,"{'word': '유회승', 'start_idx': 61, 'end_idx': 63...","{'word': '엔플라잉', 'start_idx': 56, 'end_idx': 5...",per:employee_of,wikitree
24004,24027,"유럽 위원회 의장 조제 마누엘 바호주, 아르메니아 대통령 세르지 사르키샨, 에스토니...","{'word': '세르지 사르키샨', 'start_idx': 32, 'end_idx...","{'word': '아르메니아', 'start_idx': 22, 'end_idx': ...",per:origin,wikipedia
24005,24028,"대한민국에서는 1990년에 금성사(현, LG전자)에서 비디오판으로 닌자 거북이라는 ...","{'word': '금성사', 'start_idx': 15, 'end_idx': 17...","{'word': 'LG전자', 'start_idx': 22, 'end_idx': 2...",org:alternate_names,wikipedia
...,...,...,...,...,...,...
32419,32466,"법포는 다시 최시형, 서병학, 손병희 직계인 북접과 다시 서장옥, 전봉준, 김개남을...","{'word': '최시형', 'start_idx': 7, 'end_idx': 9, ...","{'word': '손병희', 'start_idx': 17, 'end_idx': 19...",per:colleagues,wikipedia
32420,32467,완도군(군수 신우철)이 국토교통부에서 실시한 '2019 교통문화지수 실태조사'에서 ...,"{'word': '완도군', 'start_idx': 0, 'end_idx': 2, ...","{'word': '신우철', 'start_idx': 7, 'end_idx': 9, ...",org:top_members/employees,wikitree
32421,32468,"중앙일보, JTBC 회장을 지낸 이후 중앙홀딩스 회장, 재단법인 한반도평화만들기 이...","{'word': 'JTBC', 'start_idx': 6, 'end_idx': 9,...","{'word': '중앙홀딩스', 'start_idx': 21, 'end_idx': ...",no_relation,wikipedia
32422,32469,화순군(군수 구충곤)은 17일 동면의 이장 20여 명이 코로나 19 예방을 위해 버...,"{'word': '화순군', 'start_idx': 0, 'end_idx': 2, ...","{'word': '구충곤', 'start_idx': 7, 'end_idx': 9, ...",org:top_members/employees,wikitree


In [10]:
trans_list=[]
for i, sen, sub, obj, label, source in notebook.tqdm(zip(df['id'], df['sentence'], df['subject_entity'], df['object_entity'], df['label'], df['source'])):
    #no_relation이 아니면서 / 외국어가 전혀 없으면서 / 각 엔티티가 문장에 한 번만 존재하는 경우에
    if( label!="no_relation"):
        if(find_foreign('4e00', '9fff', sen)==False) and (find_foreign('0400', '04ff', sen)==False) and (find_foreign('0600', '06ff', sen)==False) and (find_foreign('0370', '03ff', sen)==False) and (find_foreign('3040', '309f', sen)==False) and (find_foreign('30a0', '30ff', sen)==False):
            sub = eval(sub)
            obj = eval(obj)
    
            sub_word=sub['word']
            obj_word=obj['word']
    
            sub_word_cnt=sen.count(sub_word)
            obj_word_cnt=sen.count(obj_word)
            
            if(sub_word_cnt==1 and obj_word_cnt==1): #역번역하자! #17658개에 대해서
                new_sen=back_trans(sen,"en")
                # new_sen=back_translation(sen,"en")
                if sub_word in new_sen and obj_word in new_sen: #번역시에도 문장의 엔티티가 잘 살아있는 경우에만 추가
                    ss=new_sen.find(sub_word)
                    se=ss+len(sub_word)-1
                    os=new_sen.find(obj_word)
                    oe=os+len(obj_word)-1
                    
                    sub['start_idx']=ss
                    sub['end_idx']=se
                    obj['start_idx']=os
                    obj['end_idx']=oe
                    
                    sub=str(sub)
                    obj=str(obj)
                    
                    temp=[]
                    temp.append(i)
                    temp.append(new_sen)
                    temp.append(sub)
                    temp.append(obj)
                    temp.append(label)
                    temp.append(source)
                    trans_list.append(temp)

0it [00:00, ?it/s]

공화인민당 (CHP)는 30%선을 넘기는 것을 목표로 삼고 최상으로는 잠재적 소수 정당들의 도움을 받아 내각을 수립하거나 당 하나의 힘으로 수립하는 것을 목표로 삼았다.
이소연 한국투자증권 연구원은 “BID는 2017년부터 전략적 투자자를 물색해왔다. 현재 주주는 SBV 95.3%, 외국인 2.3%, 기타 2.4%로 구성되어 있다. 이후 2020년까지 SBV 65%, 전략적 투자자로서의 하나은행 15%, 외국계 재무적 투자자 12%, 기타 5%로 지분구조가 변화될 예정”이라며 “이번 투자유치 배경은 내년부터 베트남에서 적용될 바젤 II에 있다며 BID의 총자본비율은 지난해 말 기준 10.3%인데, 바젤 II가 적용되면 약 2%p가 하락할 것으로 추정된다”고 설명했다.
비례대표 정당투표 조사에서는 민주당을 선택하겠다는 응답이 37.7%, 미래통합당 비례대표용 위성정당인 미래한국당을 선택하겠다는 29.2% 오른 29.2%였다.
그리고 청소년들의 사전 수요조사를 통해 선정한 가수로 최근 TV드라마 호텔 델루나의 OST를 부른 십센치(10cm)[권정열]가 축하공연에서 달콤한 음성으로 가을 밤하늘의 운치를 더할 계획이다.
유승민 바른미래당 전 대표 4.0%, 안철수 바른미래당 전 의원 3.6%, 심상정 정의당 대표 3.3%, 김경수 경남지사 3.1%, 나경원 한국당 원내대표 3.0%, 박원순 서울시장 3.0% 등이 3.0% 이상의 선호도를 기록했다.
[[1945년]] [[8월 16일]] [[백남훈]], [[김성수 |김성수]], [[김도연 |김도연]], [[송진우 |송진우]], [[윤보선]], [[허정]], [[장덕수]] 등과 함께 정치 세력을 형성한 뒤 [[우익]] 세력 연합으로 9월 [[한국민주당]]을 창당한데 참여한 것을 시작으로 본격적인 정치 활동을 시작했고, [[미군정]] 시기에 [[경기도 도지사]]를 역임했다.
2006년 아이티 국민들은 의회를 뽑았고, 2월에 대통령 선거를 통해 르네 프레발이 51%의 득표율로 당선하였고, 5월에 대통령으로 취임했다.
최정욱 

In [14]:
new_df = pd.DataFrame(trans_list)
new_df
new_df.to_csv('/opt/ml/project2/level2_klue_nlp-level2-nlp-01/dataset/train/back_translation_output_eng.csv',index=False)

## entity가 문장에 없으면 출력

In [15]:
for sen, sub, obj in zip(df['sentence'], df['subject_entity'], df['object_entity']):
    sub = eval(sub)
    obj = eval(obj)
    sub_word=sub['word']
    obj_word=obj['word']
    if sub_word not in sen:
        print(sen)
        print(sub_word)
    if obj_word not in sen:
        print(sen)
        print(obj_word)

## index에 문제 있으면 출력

In [16]:
for sen, sub, obj in zip(df['sentence'], df['subject_entity'], df['object_entity']):
    raw_sub = eval(sub)
    raw_obj = eval(obj)
    ss=raw_sub['start_idx']
    se=raw_sub['end_idx']
    os=raw_obj['start_idx']
    oe=raw_obj['end_idx']
    if(sen[ss:se+1]!=raw_sub['word']):
        print(sen[ss:se+1])
        print(raw_sub['word'])
        
    if(sen[os:oe+1]!=raw_obj['word']):
        print(sen[os:oe+1])
        print(raw_obj['word'])